In [8]:
import json
from models import Appointment, YearParty
from database_utils import Session; session = Session();
from collections import Counter, defaultdict
from sqlalchemy import sql


In [167]:
def get_judge_count_query(session):
    join_conditions = [
        # Join condition for if the judge was serving that year
        sql.and_(
            Appointment.start_year < YearParty.year + 2,
            sql.or_(
                Appointment.end_year >= YearParty.year, Appointment.end_year.is_(None)
            )
        ),
        # Join condition for party
        YearParty.party == Appointment.party_of_appointing_president,
    ]


    return (
        session
        .query(
            YearParty.year,
            YearParty.party,
            Appointment.court_type,
            Appointment.court_name,
            sql.func.count(Appointment.start_year).label('count'),
        )
        .outerjoin(
            Appointment,
            sql.and_(*join_conditions)
        )
        .group_by(YearParty.year, YearParty.party, Appointment.court_type, Appointment.court_name)
        .order_by(YearParty.year)
    )


def get_start_count_query(session):
    join_conditions = [
        # Need to have started / left in that congress or after
        Appointment.start_year >= YearParty.year,
        # Need to have started /  before the start of the next congress
        Appointment.start_year < YearParty.year + 2,
        # Join condition for party
        YearParty.party == Appointment.party_of_appointing_president,
    ]

    return (
        session
        .query(
            YearParty.year,
            YearParty.party,
            Appointment.court_type,
            Appointment.court_name,
            sql.func.count(Appointment.start_year).label('count'),
        )
        .outerjoin(
            Appointment,
            sql.and_(*join_conditions)
        )
        .group_by(YearParty.year, YearParty.party, Appointment.court_type, Appointment.court_name,)
        .order_by(YearParty.year)
    )


def get_end_count_query(session):
    join_conditions = [
        # Need to have started / left in that congress or after
        Appointment.end_year >= YearParty.year - 2,
        # Need to have started /  before the start of the next congress
        Appointment.end_year < YearParty.year,
        # Join condition for party
        YearParty.party == Appointment.party_of_appointing_president,
    ]

    return (
        session
        .query(
            YearParty.year,
            YearParty.party,
            Appointment.court_type,
            Appointment.court_name,
            sql.func.count(Appointment.start_year).label('count'),
        )
        .outerjoin(
            Appointment,
            sql.and_(*join_conditions)
        )
        .group_by(YearParty.year, YearParty.party, Appointment.court_type, Appointment.court_name,)
        .order_by(YearParty.year)
    )


In [116]:
start_query = get_start_count_query(session)
end_query = get_end_count_query(session)
count_query = get_judge_count_query(session)


In [160]:
super_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
court_name_to_court_type = {}

for row in count_query:
    super_dict[row.year][row.court_name][row.party]["n_judges"] = row.count
    court_name_to_court_type[row.court_name] = row.court_type
    
for row in end_query:
    super_dict[row.year][row.court_name][row.party]["n_terminated"] = row.count
    court_name_to_court_type[row.court_name] = row.court_type

for row in start_query:
    super_dict[row.year][row.court_name][row.party]["n_appointed"] = row.count
    court_name_to_court_type[row.court_name] = row.court_type

In [161]:
flat_super_dict_rows = []
for year, court_dict in super_dict.items():
    for court_name, party_dict in court_dict.items():
        court_type = court_name_to_court_type[court_name]
        for party, count_dict in party_dict.items():
            flat_super_dict_rows.append({
                "year": year,
                "court_name": court_name,
                "court_type": court_type,
                "party": party,
                "n_terminated": count_dict.get("n_terminated", 0),
                "n_appointed": count_dict.get("n_appointed", 0),
                "n_judges": count_dict.get("n_judges", 0)
            })

In [168]:
json.dump(flat_super_dict_rows, open("../data/flat_court_party_judge_counts.json", "w"))

In [166]:
year = 1955
court_name = 'U.S. Court of Appeals for the District of Columbia Circuit'
court_type = None

filtered_rows = []
for row in flat_super_dict_rows:
    if year and row['year'] != year:
        continue
    if court_name and row['court_name'] != court_name:
        continue
    if court_type and row['court_type'] != court_type:
        continue
        
    filtered_rows.append(row)

print(sum(
    x["n_judges"] for x in filtered_rows
    if x["party"] == 'Republican'
))
print(sum(
    x["n_judges"] for x in filtered_rows
    if x["party"] == 'Democratic'
))

print(sum(
    x["n_appointed"] for x in filtered_rows
    if x["party"] == 'Republican'
))
print(sum(
    x["n_appointed"] for x in filtered_rows
    if x["party"] == 'Democratic'
))
print(sum(
    x["n_terminated"] for x in filtered_rows
    if x["party"] == 'Republican'
))

print(sum(
    x["n_terminated"] for x in filtered_rows
    if x["party"] == 'Democratic'
))

3
6
1
0
0
2


162